In [1]:
import pandas as pd
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# récupérer les fichier du dossier contenant les fihcier de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_cluster= glob.glob('/content/drive/MyDrive/Colab Notebooks/km/*clean*')

In [4]:
# vérifier la liste
print(list_cluster)

['/content/drive/MyDrive/Colab Notebooks/km/km_cluster_2_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_1_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_0_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_5_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_6_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_9_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_8_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_3_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_7_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km/km_cluster_4_clean_msg.txt']


In [5]:
# fonction pour lire une liste de fichier 
# argument : une liste de fichiers
# sortie: dictionnaire
def read_cluster(files):
  dic={}
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      dic[i] = file.readlines()
  return dic

In [6]:
dic= read_cluster(list_cluster)

In [7]:
print(len(dic[9]))

184481


In [8]:
# Mettre chaque dictionnaire dans un dataframe
for i in range(len(list_cluster)):
  vars()["data_"+str(i)]=pd.DataFrame(dic[i], columns = ['tweet'])

In [9]:
print(data_3.head(10))

                 tweet
0                   \n
1                   \n
2                   \n
3                   \n
4                   \n
5  alexanderducharme\n
6         ampkkqvdid\n
7       audreyloggin\n
8       beautybashhh\n
9       beautybashhh\n


In [ ]:
!pip install gensim==4.0

In [11]:
from gensim.models import KeyedVectors

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [12]:
# récupérer les embeddings entrainés sur nos données
w2v = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/w2vec_model_d300_1M')

In [13]:
import numpy as np

In [14]:
#encode un message
def encode(msg):
    return np.mean([w2v[word] for word in msg if word in w2v] or [np.zeros(300)], axis = 0)

In [15]:
w2v.similar_by_word("sale")

[('sal', 0.6782066822052002),
 ('pvte', 0.629138708114624),
 ('mèrde', 0.6162537336349487),
 ('pute', 0.6096962094306946),
 ('pte', 0.6044666767120361),
 ('clochard', 0.6014346480369568),
 ('slp', 0.6004031896591187),
 ('crasseux', 0.5957321524620056),
 ('puent', 0.5908592939376831),
 ('daronn', 0.5900101661682129)]

In [16]:
# Encoder un dictionnaire 
def encoded(dic):
  w2v={}
  for i in range(len(dic)):
    w2v[i]=np.array([encode(msg) for msg in dic[i]])
  return w2v

In [17]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded = encoded(dic)

CPU times: user 45.5 s, sys: 3.15 s, total: 48.6 s
Wall time: 48.8 s


In [18]:
print(len(dataset_encoded[0]))

4333


In [23]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [24]:
# Entrainer le modèle
%%time
for i in range(len(dataset_encoded)):
  clf.fit(dataset_encoded[i])

CPU times: user 8min 55s, sys: 2.99 s, total: 8min 58s
Wall time: 4min 44s


In [25]:
#mettre les prédictions dans un dictionnaire
%%time
pred={}
for i in range(len(dataset_encoded)):
  pred[i] = clf.predict(dataset_encoded[i])

CPU times: user 2min 9s, sys: 5.12 s, total: 2min 14s
Wall time: 2min 23s


In [26]:
print(pred[1])

[ 1  1 -1 ...  1  1  1]


In [27]:
# ajouter une colonne pred à chaque dataframe de cluster
for i in range(len(dataset_encoded)):
  vars()["data_"+str(i)]['pred']= pred[i]

In [28]:
print(data_1.head(20))

                                                tweet  pred
0                                             roser\n     1
1                 our choice is rosé mamavote roser\n     1
2                    ra_apex_divisioncommission for\n    -1
3   manifesting surprise dm manifesting surprise d...     1
4                                              vote\n     1
5                                               vot\n     1
6                                       vote carats\n     1
7                        géministhesong butter pcer\n     1
8   votes votes votes vote votes vote come elf vot...     1
9                 themusicvideo butter pca vote bts\n     1
10                                        vote damn\n     1
11                                             vote\n     1
12                                             vote\n     1
13                                             vote\n     1
14                                             vote\n     1
15                                  vote

In [29]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
outliers={}
for i in range(len(dataset_encoded)):
  outliers[i]=vars()["data_"+str(i)].loc[vars()["data_"+str(i)]['pred']==-1]
  outlier_index=list(outliers[i].index)
  print("cluster"+str(i)+":"+"\n")
  print(vars()["data_"+str(i)]['pred'].value_counts())
  print("\n")

cluster0:

 1    4263
-1      70
Name: pred, dtype: int64


cluster1:

 1    4115
-1      68
Name: pred, dtype: int64


cluster2:

 1    79715
-1    10267
Name: pred, dtype: int64


cluster3:

 1    31015
-1      846
Name: pred, dtype: int64


cluster4:

 1    70205
-1    23295
Name: pred, dtype: int64


cluster5:

 1    13206
-1      690
Name: pred, dtype: int64


cluster6:

 1    74577
-1     6548
Name: pred, dtype: int64


cluster7:

 1    75177
-1    32094
Name: pred, dtype: int64


cluster8:

 1    45472
-1     2028
Name: pred, dtype: int64


cluster9:

 1    148950
-1     35531
Name: pred, dtype: int64




In [34]:
for i in range(len(outliers)):
  with open(f"drive/MyDrive/outliers/outliers{i}", 'w') as f:
    f.write(str(outliers[i]))